Batch creating stations in OSCAR/Surface
---
> **Example #4:** We will upload a list of stations from Brazil to OSCAR/Surface. The information about new stations is contained in an [Excel file](https://github.com/kurt-hectic/wmo-api-webinar/blob/master/files/Stations_Template-INMET.csv)  which I got from Jose Mauro, and which I converted into CSV. Before creating new stations we need to process the information in the Excel sheet.
We need to do two things. First, to rename the columns so that they correspond to the expected names. Second, to translate the values into WIGOS Metadata Standard code-list values. Finally, we upload (or write to file) the XML using the abstrction layer provided by the library.

In [3]:
# import the libraries we will be working with. See requirements.txt file for environment setup
import json
import sys
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("dicttoxml").setLevel(logging.WARNING)
logging.getLogger("oscar-lib").setLevel(logging.INFO)

import pandas as pd
import re

from oscar_lib import OscarClient, Station, OscarGUIClient

import pkg_resources  # part of setuptools
version = float(pkg_resources.require("oscar_lib")[0].version)

if version < 0.41:
    raise Exception("need at least version 0.41 of oscar client .. install with pip install git+git://github.com/kurt-hectic/oscar-client.git@develop ")

# checking the Excel sheet
This is how the Excel sheet (in CSV format) looked initially (I only output the first 3 rows)

In [4]:
df_stations_orig = pd.read_csv(r'files/Stations_Template-INMET.csv',encoding="latin1") #read the CSV file into a dataframe
df_stations_orig.head(3)

,Name,Identifier,additional_wigos_id,Type,Automatic,Latitude,Longitude,Altitude,Creation,International,...,real time,affiliations,Frequency,variables,WMO region,Country / Territory,Time zone,Station URL,Other link (URL),Site description
0,Novo Aripuanã,0-76-0-1303304000000594,0-76-1-1303304000000594,Land (Fixed),automatic,-5.141139,-60.380531,45.0,18.07.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-4,http://tempo.inmet.gov.br/WSI/0-76-0-130330400...,https://cidades.ibge.gov.br/brasil/am/novo-ari...,Automatic weather station
1,Redenção,0-76-0-1506139000000593,0-76-1-1506139000000593,Land (Fixed),automatic,-8.043250,-50.006917,199.0,17.06.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-150613900...,https://cidades.ibge.gov.br/brasil/pa/redencao...,Automatic weather station
2,Zé Doca,0-76-0-2114007000000596,0-76-1-2114007000000596,Land (Fixed),automatic,-3.269194,-45.651083,45.5,11.07.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-211400700...,https://cidades.ibge.gov.br/brasil/ma/ze-doca/...,Automatic weather station


## rename the columns
First we need to make sure the column names correspond to the names that are expected by the OSCAR client library.
For this we rename the columns according to the mapping underneath

In [5]:
df_stations_orig.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True) # remove empty rows
df_stations_orig = df_stations_orig.rename(columns=lambda x: x.strip().lower() ) #fix column names: remove potential whitespace and make lowecase
# rename the column names to expected format 
column_map = { 
        'identifier' : 'wigosid' , 
        'type' : 'stationtype' ,  
        'altitude' : 'elevation' , 
        'creation' : 'established' , 
        'operational status' : 'status',
        'real time' : 'real-time',
        'wmo region':'region',
        'country / territory' : 'country',
        'station url':'url',
        'site description' : 'description' , 
        'time zone':'timezone' }
df_stations_orig = df_stations_orig.rename(columns=column_map )
df_stations_orig.head(3)

,name,wigosid,additional_wigos_id,stationtype,automatic,latitude,longitude,elevation,established,international,...,real-time,affiliations,frequency,variables,region,country,timezone,url,other link (url),description
0,Novo Aripuanã,0-76-0-1303304000000594,0-76-1-1303304000000594,Land (Fixed),automatic,-5.141139,-60.380531,45.0,18.07.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-4,http://tempo.inmet.gov.br/WSI/0-76-0-130330400...,https://cidades.ibge.gov.br/brasil/am/novo-ari...,Automatic weather station
1,Redenção,0-76-0-1506139000000593,0-76-1-1506139000000593,Land (Fixed),automatic,-8.043250,-50.006917,199.0,17.06.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-150613900...,https://cidades.ibge.gov.br/brasil/pa/redencao...,Automatic weather station
2,Zé Doca,0-76-0-2114007000000596,0-76-1-2114007000000596,Land (Fixed),automatic,-3.269194,-45.651083,45.5,11.07.2019,yes,...,yes,GOS,1 hour,"Air Temperature (inst, max, min), Relative Hum...",III - South America,Brazil,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-211400700...,https://cidades.ibge.gov.br/brasil/ma/ze-doca/...,Automatic weather station


## map content to WIGOS metadata record values
We need to make sure the values in the Excel sheet correspond to the WIGOS metadata standard codelists in https://codes.wmo.int/_wmdr . We need to translate the values in the following fields: **stationtype, international, real-time, region, country** and **variables**. 

In [6]:
# helper functions to parse the variables and frequency
def parseFrequency(x):
    """ Extract schedule and return as seconds
    Currently only hours are supported
    """
    m=re.search('(\d+)\s+hour',x)
    m2=re.search('(\d+)\s+minutes',x)
    if m:
        return int(m.group(1)) * 60 * 60 # we can only recognize hours at the moment in this code
    elif m2:
        return int(m.group(1)) * 60
    else:
        return None

var_map = {
    'Air Temperature':224,
    'Relative Humidity':251,
    'Dewpoint':225,
    'Atmospheric Pressure':216,
    'Winds':[12005,12006],
    'Solar Radiation':572,
    'Precipitation':210
}
    
def parseVariables(variables):
    """ Extract variables from Excel sheet and return them as numeric WMDR codelist values"""
    res = []
    for v in re.sub('\(.+?\)','',variables).split(","):
        v=v.strip()
        tmp = var_map[v]
        if not isinstance(tmp, list):
            tmp = (tmp,)
        res.extend( tmp )
        
    return res

Here we do the actual mapping of the values. The [WMO codes registry](https://codes.wmo.int/_wmdr) contains the codelist values. Underneath, I only provide mappings for the values used in our Excel sheet.

In [7]:
df_stations = df_stations_orig.copy()

# here we map the values in the sheet to the WMDR values.
type_map = { 'Land (Fixed)' : 'landFixed' }
automatic_map = {'automatic':'automaticReading'}
status_map = {'operational':'operational'}
country_map = {'Brazil':'BRA'}
region_map = {'III - South America':'southAmerica'}

# perform the mapping 
df_stations["stationtype"] = df_stations["stationtype"].map(type_map) 
df_stations["automatic"] = df_stations["automatic"].map(automatic_map) 
df_stations["status"] = df_stations["status"].map(status_map) 

df_stations["international"] = df_stations["international"].astype(bool)
df_stations["real-time"] = df_stations["real-time"].astype(bool)

df_stations["region"] = df_stations["region"].map(region_map)
df_stations["country"] = df_stations["country"].map(country_map)

df_stations["frequency"]=df_stations["frequency"].apply(parseFrequency).astype(int)
df_stations["variables"]=df_stations.variables.map(parseVariables)

df_stations["established"] = pd.to_datetime(df_stations["established"]).dt.date

df_stations.head(3)

,name,wigosid,additional_wigos_id,stationtype,automatic,latitude,longitude,elevation,established,international,...,real-time,affiliations,frequency,variables,region,country,timezone,url,other link (url),description
0,Novo Aripuanã,0-76-0-1303304000000594,0-76-1-1303304000000594,landFixed,automaticReading,-5.141139,-60.380531,45.0,2019-07-18,True,...,True,GOS,3600,"[224, 251, 225, 216, 12005, 12006, 572, 210]",southAmerica,BRA,UTC-4,http://tempo.inmet.gov.br/WSI/0-76-0-130330400...,https://cidades.ibge.gov.br/brasil/am/novo-ari...,Automatic weather station
1,Redenção,0-76-0-1506139000000593,0-76-1-1506139000000593,landFixed,automaticReading,-8.043250,-50.006917,199.0,2019-06-17,True,...,True,GOS,3600,"[224, 251, 225, 216, 12005, 12006, 572, 210]",southAmerica,BRA,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-150613900...,https://cidades.ibge.gov.br/brasil/pa/redencao...,Automatic weather station
2,Zé Doca,0-76-0-2114007000000596,0-76-1-2114007000000596,landFixed,automaticReading,-3.269194,-45.651083,45.5,2019-11-07,True,...,True,GOS,3600,"[224, 251, 225, 216, 12005, 12006, 572, 210]",southAmerica,BRA,UTC-3,http://tempo.inmet.gov.br/WSI/0-76-0-211400700...,https://cidades.ibge.gov.br/brasil/ma/ze-doca/...,Automatic weather station


## creating Station objects and exporting the XML or uploading it to OSCAR/Surface
Finally we loop over the stations in the table and create a Station object for each row. The Station object is an abstraction layer to the XML and allows to write the XML into a file, or to upload it using an oscar client object.

Most parameters can be directly passed into the constructor of the Station object. But for the nested structure of observations and schedules we need to preproces the information.

Finally, there is some custom mapping. Jose Mauro told me to represent the manufacturer as text in the site description. So I add this field to the field _description_ . I also set the _superivsing organiaztion_ to "INMET". 

In [8]:
# a client object for the interaction with OSCAR/Surface
client = OscarClient(oscar_url = OscarClient.OSCAR_DEPL, token="changeMe")

In [9]:
# default schedule . We overwrite values with values from the Excel sheet where available
default_schedule = {   
   "startMonth": 1,   "endMonth": 12,
   "startWeekday": 1, "endWeekday": 7,
   "startHour": 0,    "endHour": 23,
   "startMinute": 0,  "endMinute": 59,
   "interval": 60*60, "international": True , "real-time" : True # we overwrite these with the values from Excel
}

# loop over the rows in the Excel
for idx,row in df_stations[df_stations["in oscar"] == "yes"].iterrows():  
    # most parameters needed in the constructor of Station are already in the dictionary, as we renamed the columns above
    params = dict(row)
    
    #adding custom information
    params["organization"] = "INMET" # I looked up this abbreviation from the field "supervising organization" in the station search dialogue in OSCAR
    params["urls"] = [params["url"], params["other link (url)"]] # pass multiple urls as a lisst
    params["description"] = params["description"] + ". Station manufacturer is " + params["manufacturer"] # we integrate the manufactures in the site description 
    
    # we need to take care of the observations and schedules
    observations = []
    for v in row["variables"]: # loop over the variables. Our extraction function above already returned a list
        new_schedule = default_schedule.copy()
        new_schedule["international"] = row["international"]
        new_schedule["interval"] = row["frequency"]
        new_schedule["real-time"] = row["real-time"]
        
        # construct the observation part. We also assign the schedule we created above
        observation = { 
            "variable" : v , "observationsource" : params["automatic"] , 
            "affiliation": params["affiliations"] , "schedule" : new_schedule 
        }
        observations.append(observation)
        
    params["observations"] = observations
        
    s = Station(**params) # create the station
        
    try:
        s.validate() # validate the result against the WMDR schema
        
        # write XML to disk
        with open(r"tmp/{}.xml".format(row["wigosid"]),"w",encoding="utf8") as f: 
            f.write(str(s))
            
        # upload to OSCAR
        status=client.upload_XML(str(s))
        print("uploaded {}, {}".format(row["wigosid"],status))
        
        break # only process one station

    except Exception as e:
        print("error:",e)


INFO:oscar_lib.oscar_client:upload ok, new id 201894 The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).
# Facility with identifier "0-76-0-1303304000000594"
REF_4: An entry for "facility/geospatialLocation" with the same beginPosition = 2019-07-18 already exists. Information is discarded.
REF_4: An entry for "facility/territory" with the same beginPosition = 2019-07-18 already exists. Information is discarded.
REF_4: An entry for "facility/description/Description/description" with the same beginPosition = 2019-07-18 already exists. Information is discarded.
REF_4: The time period with "beginPosition" = 2019-07-18 overlaps with the time period of an existing status. "facility/programAffiliation/ProgramAffiliation/reportingStatus/ReportingStatus" is discarded.

INFO:oscar_lib.oscar_client:status: VALID_XML_WITH_ERRORS_OR_WARNINGS message: upload ok, new id 2

uploaded 0-76-0-1303304000000594, VALID_XML_WITH_ERRORS_OR_WARNINGS


## Adding additional WIGOS IDs and the Timezone
The current version of the WIGOS metadata XML schema does not support multiple IDs. It also does not support the timezone element of the WIGOS metadata standard.
We use the OSCAR API to set these two additional fields at all stations.

***Note***: This uses the internal OSCAR API and may not always work, as updates to the GUI also affect this API. Make sure you have installed the latest version of the oscar_lib.

In [6]:
df_stations[["wigosid","additional_wigos_id","timezone"]].head(2)

,wigosid,additional_wigos_id,timezone
0,0-76-0-1303304000000594,0-76-1-1303304000000594,UTC-4
1,0-76-0-1506139000000593,0-76-1-1506139000000593,UTC-3


In [7]:
# internal client.. need to supplie OSCAR username and password instead of token
gui_client = OscarGUIClient(OscarClient.OSCAR_DEPL,username="tproescholdt@wmo.int",password="mypw")

In [9]:
logging.getLogger("oscar_lib.oscar_gui_client").setLevel(logging.INFO)

# we loop again over the list of stations
for idx,row in df_stations[df_stations["in oscar"] == "yes"].iterrows():  
    # most parameters needed in the constructor of Station are already in the dictionary, as we renamed the columns above
    params = dict(row)
    
    wigos_id = params["wigosid"]
    additional_wigos_id = params["additional_wigos_id"]
    timezone = params["timezone"]
    established = str(params["established"])
    
    internal_id = gui_client.wigos_to_internal_id(wigos_id) # obtain internal id of the station using existing wigos id
    print("internal id:",internal_id)
    
    print("add wigos id:",additional_wigos_id,gui_client.add_wigos_id(internal_id,additional_wigos_id,primary=False)) # set additional wigos id
    print("add timezone:",timezone,gui_client.add_timezone(internal_id,timezone,valid_since=established)) # set timezone

    break # only process one station for demo purposes


internal id: 56827
add wigos id: 0-76-1-1303304000000594 500
add timezone: UTC-4 204
